# Softmax Regression

softmax模型可以用来给不同的对象分配概率。<br>
在训练更加复杂的模型时，最后一步也往往需要用softmax来分配概率。<br>

softmax回归分两步：
* 首先 对某个待分类对象属于某个类的“证据”相加求和
* 然后 将这个“证据”的和和转化为概率

例如计算一张图片是否属于某类。<br>
使用加权的方法来累积计算这张图是否属于某类的“证据”。<br>
如果图片的像素强有力的体现该图不属于某个类，则权重的数值为负数。<br>
相反，则权重的数值为正。<br>
还需要引入额外的“证据”，称之为偏置量(bias)<br>
因此对于给定的输入图片$x$是属于第$i$类的总体“证据”可以表示为：

$$
evidence_i = \sum_{i}W_{i,j}x_j + b_i
$$

其中$W_i$代表权重，$b_i$代表第$i$类的偏置量，$j$代表给定图片$x$的像素索引 用于求和<br>
然后利用softmax函数可以把这些“证据”转化成概率$y$

$$
y = softmax(evidence)
$$

给定一张图$x$，它对于每一个类别的吻合度可以被softmax函数转换成一个概率值<br>
softmax函数可以定义为：

$$
softmax(x) = normalize(exp(x))
$$

展开右边可以得到：

$$
softmax(x)_i = \frac{exp(x_i)}{\sum_{j}exp(x_j)}
$$

假设模型里的权值不可以是0或是负数，softmax会正则化这些权重值，<br>
是它们的总和等于1，以此构建一个有效率的概率分布。

$$
\left[
\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}
\right] = softmax\left(
\begin{matrix}
W_{1,1}x_1 + W_{1,2}x_1 + W_{1,3}x_1 + b_1 \\
W_{2,1}x_2 + W_{2,2}x_2 + W_{2,3}x_2 + b_2 \\
W_{3,1}x_3 + W_{3,2}x_3 + W_{3,3}x_3 + b_3
\end{matrix}
\right)
$$

转换成用 矩阵乘法和向量相加来表示：

$$
\left[
\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}
\right] = softmax\left(
\left[\begin{matrix}
W_{1,1}&W_{1,2}&W_{1,3} \\
W_{2,1}&W_{2,2}&W_{2,3} \\
W_{3,1}&W_{3,2}&W_{3,3}
\end{matrix}\right]
\cdot
\left[\begin{matrix}
x_1 \\
x_2 \\
x_3
\end{matrix}\right]
+
\left[\begin{matrix}
b_1 \\
b_2 \\
b_3
\end{matrix}\right]
\right)
$$

简化为：
$$
y = softmax(W_x+b)
$$

## 评估模型
为了训练模型，通常需要定义一个指标来评估模型的好坏。<br>
这个指标被称为成本(cost)或是损失(loss)，两者意思相同，然后尽量最小化这个指标。<br>
非常常见的成本函数是“交叉熵”(cross-entropy):

$$
H_{y'}(y) = -\sum_{i}y'_i\log(y_i)
$$

其中y是预测的概率分布。<br>
y‘是实际的分布。<br>

## 计算交叉熵
（重要）<br>
交叉熵不仅仅用来衡量单一的一对预测和真实值，<br>
而是所有图片的交叉熵的总和。<br>
对100个数据点的预测的表示比单一数据点的预测的表示能更好的描述模型的性能。

In [3]:
import tensorflow as tf
import input_data
import numpy as np
import time
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
np.set_printoptions(suppress=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
### 参数设置----------------------------------

# MNIST数据的图片特征个数 28x28=784
n_input = 784

# MNIST数据的种类个数 0-9
n_classes = 10

# 学习速率，比较好的策略是先设置为0.25
# 然后在训练到第20个Epoch时改为0.025
# 学习速率太大时会导致代价函数振荡
# 学习速率太小时会导致收敛过慢
learning_rate = 0.25

# 批尺寸mini-batch
# 太大，权重的更新就不会那么频繁，优化过程太漫长
# 太小，计算的加速效果越不明显
batch_size = 100

# 训练数据55000个，每次取100，学习一个epoch需要550次
total_batch = int(mnist.train.num_examples / batch_size)

# 单次训练迭代
# 1个epoch意味着训练数据被用过一遍
training_epochs = 10

### 创建模型----------------------------------

# 创建一个占位符x来保存输入值
# None表示第一个维度可以是任何长度（输入图片的个数）
# 第二个维度784=28x28是每一个图片的展平后的向量
x = tf.placeholder(tf.float32, [None, n_input])

# 创建一个占位符来保存权重值W和偏置量b
# Variable常用来保存参数，计算中可以被修改，初始值为0
# 0~9共10个类别
W = tf.Variable(tf.zeros([n_input, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))

# y是预测出来的分类结果
# x乘以W(相当于W_x)再加上b，然后输入到softmax里面
y_evidence = tf.matmul(x, W) + b
y_softmax = tf.nn.softmax(y_evidence)

### 计算交叉熵----------------------------------

# 创建一个占位符y_true用来保存输出的正确值（正确分类）
y_true = tf.placeholder(tf.float32, [None, n_classes])

# 计算交叉熵，又叫cost
# 直接按照公式写出的代码，如果出现log(0)的话结果就会变成NaN，不建议使用！！！！！
# cross_entropy = tf.reduce_mean(
#     -tf.reduce_sum(y_true * tf.log(y_softmax), reduction_indices=[1]))
# cross_entropy = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(
#         labels=y_true, logits=y_evidence))
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y_true, logits=y_evidence))

# 学习步伐，又叫optimizer
# 优化方法使用梯度下降算法（gradient descent algorithm）来最小化交叉熵
# 会自动使用反向传播法（backpropagation algorithm）
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
    cross_entropy)

### 训练和评估模型----------------------------------

# 建立一个交互式的会话
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# 评估
pred = tf.argmax(y_evidence, 1)
true = tf.argmax(y_true, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 循环训练10次
# 每次训练会随机抓取训练数据中100个数据点
start = time.time()
for epoch_i in range(training_epochs):
    ave_cost = 0
    for batch_i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run(
            [train_step, cross_entropy],
            feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
        ave_cost += c / total_batch
        # evidence and softmax data
        if (epoch_i == 0 and batch_i == 0) or (epoch_i == 9 and batch_i == 0):
            train_evidence = y_evidence.eval(feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
            train_softmax = y_softmax.eval(feed_dict={
                x: batch_xs,
                y_true: batch_ys
            })
            print("train_evidence:", train_evidence[:5])
            print("train_softmax:", train_softmax[:5])
            train_pred = pred.eval(feed_dict={x: batch_xs, y_true: batch_ys})
            train_true = true.eval(feed_dict={x: batch_xs, y_true: batch_ys})
            test_pred = pred.eval(feed_dict={
                x: mnist.test.images,
                y_true: mnist.test.labels
            })
            test_true = true.eval(feed_dict={
                x: mnist.test.images,
                y_true: mnist.test.labels
            })
            print("train_pred:", train_pred[:5])
            print("train_true:", train_true[:5])
            print("test_pred:", test_pred[:5])
            print("test_true:", test_true[:5])
    if epoch_i % 1 == 0:
        train_acc = accuracy.eval(feed_dict={x: batch_xs, y_true: batch_ys})
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        })
        print("Epoch:%3d Batch:%5d " % (epoch_i,
                                        batch_i), "train_acc=%.5f" % train_acc,
              "test_acc=%.5f" % test_acc, "train_cost=%.9f" % ave_cost)
end = time.time()
print("Process Time(s):{:.2f}".format(end - start))

### 输出结果----------------------------------
print(
    sess.run(
        accuracy, feed_dict={
            x: mnist.test.images,
            y_true: mnist.test.labels
        }))
sess.close()

train_evidence: [[ 0.07236662 -0.2547763   0.47817934  0.36457312 -0.37768954 -0.03963375
   0.11235216 -0.32224882  0.06431773 -0.09744085]
 [-0.07531018 -0.06043153  0.29493076  0.9898521  -0.54974014 -0.0940906
   0.02961185 -0.384664   -0.10485988 -0.04529857]
 [-0.1617511   0.46108305  0.13652168  0.21093437 -0.33643043 -0.11938936
   0.07551676 -0.2153756  -0.03320308 -0.0179065 ]
 [-0.16123295 -0.21782213  0.204731    0.41212326 -0.36740094 -0.2443259
  -0.0011595   0.11305089 -0.01292079  0.27495688]
 [-0.10842458 -0.48960537  0.9186817   0.6122737  -0.69336426 -0.21188076
   0.3144052  -0.39276838 -0.06958684  0.12026875]]
train_softmax: [[0.10370149 0.07476676 0.1556063  0.13889565 0.06611927 0.09271368
  0.10793205 0.06988849 0.10287014 0.08750614]
 [0.08464596 0.08591481 0.12257442 0.24558416 0.05267008 0.08307111
  0.09400984 0.06212346 0.0821813  0.08722483]
 [0.08296134 0.15465686 0.11179284 0.12042897 0.06966481 0.08655122
  0.10517678 0.07862975 0.09434164 0.09579585]
